In [1]:
%run load_environment.ipynb

In [2]:
import os
import pyspark.sql.functions
from pyspark.sql import SparkSession

In [3]:
appName = "NemesysNotebook"
try:
    appName = APP_NAME
except:
    pass

In [4]:
spark = (SparkSession.builder
         .appName(appName)
         .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
         .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")    
         .getOrCreate()
)
         # .enableHiveSupport()

In [5]:
#--------------------------------------------------------------------------
# Configurar o acesso ao Azure Data Lake (Blob)
#--------------------------------------------------------------------------
spark.conf.set(f'fs.azure.account.auth.type.{STORAGE_ACCOUNT}.dfs.core.windows.net', 'SharedKey')
spark.conf.set(f'fs.azure.account.key.{STORAGE_ACCOUNT}.dfs.core.windows.net', STORAGE_KEY)

In [6]:
#--------------------------------------------------------------------------
# Checar se Delta Table existe
#--------------------------------------------------------------------------
def delta_exists(delta_path, topic, db, storage_account=None, storage_key=None):
    if 'abfss://':
        url = f'DefaultEndpointsProtocol=https;AccountName={storage_account};AccountKey={storage_key};EndpointSuffix=core.windows.net'
        blob = BlobClient.from_connection_string(conn_str=url, container_name=BLOB_CONTAINER, blob_name=f'{LAKEHOUSE_PATH}/bronze/{db}_{topic}')
        return blob.exists()
    else:
        return path.exists(delta_path)
#--------------------------------------------------------------------------
# Retornar os caminhos da tabela
#--------------------------------------------------------------------------    
def paths(topico: str, db: str, tier: str):
    delta_path = f'abfss://{BLOB_CONTAINER}@{STORAGE_ACCOUNT}.dfs.core.windows.net/{LAKEHOUSE_PATH}/{tier}/{db}_{topico}'
    checkpoint_path = f'abfss://{BLOB_CONTAINER}@{STORAGE_ACCOUNT}.dfs.core.windows.net/{LAKEHOUSE_PATH}/{tier}/checkpoint/kafka/{db}_{topico}'

    return delta_path, checkpoint_path
#--------------------------------------------------------------------------
# Carregar uma delta table e registrar como temporária
#--------------------------------------------------------------------------
def loadAndRegister(table:str, db:str, tier:str = "bronze"):
    delta_path, _ = paths(table, db, tier)
    df = spark.read.format("delta").load(delta_path)
    df.createOrReplaceTempView(f"{tier}_{table}")
    return df